# Redis

On this page I'm getting to know reddis. It's a inmemory no-sql database.

## Using command line

Just use the `redis-cli` command to go into its shell. There is a `ping` command that answers `PONG` to check if everything is OK.

The following cell shows how this can look like.

In [1]:
%%bash
redis-cli
ping

PONG


## String

Is atomic thing in `redis`. There are such commands to operate on it.

|Command|What it does|
|-------|------------|
| `set` | Sets value for the variable|
| `get` | Get value from the key value|
| `del` | Delet value under key|

In [2]:
%%bash
redis-cli

set hello world
get hello

OK
world


After activating `redis-cli` we add a new key/value combination - it returns `OK` and then try to access the value under the `hello` key - it returns `world`, which is the value corresponding to the `hello` key.

Ok now lets try to delete just created key/value pair from redis.

In [3]:
%%bash
redis-cli

del hello
del hello
get hello

1
0



The first execution of `del` returns `1` because the key `hello` exists, but the second `del` returns `0` because the key `hello` has already been deleted. And as proof that the key has been deleted, we try to access it with `get` and get an empty response.

## Lists

Redis has a `list` data structure. There are some operations you can perform on redis lists.

|Command|What it does|
|-------|------------|
|`rpush`|Pushes the value onto the right end of the list|
|`lrange`|Fetches a range of values from the list|
|`lindex`|Fetches an item at a given position in the list|
|`lpop`|Pops the value from the left end of the list and returns it|

Now let's play around with it.

Here some values are pushed to the `list-key`.

In [4]:
%%bash
redis-cli

rpush list-key my
rpush list-key name
rpush list-key Fedor

1
2
3


By using `list-key` we can get values that were stored in the list.

In [5]:
%%bash
redis-cli
lrange list-key 0 -1

my
name
Fedor


By using `lindex` we can access any element you want.

In [6]:
%%bash
redis-cli

lindex list-key 0
lindex list-key 2
lindex list-key 1

my
Fedor
name


Now we can `rpop` items from the list. Each pop returns the last value of the list and clears it.

In [7]:
%%bash
redis-cli

rpop list-key
rpop list-key
rpop list-key

Fedor
name
my


## Sets

Redis has a `set` data structure - it's a bunch of unique strings. There are some commands to use them with redis sets:

|Command|What it does|
|-------|------------|
|`sadd`|Adds an item to the set|
|`smembers`|Returns entire set of items|
|`sismember`|Checks is an item is in set|
|`srem`|Remove the item from the set|

Next cells represents some experiments with this set of commands.

Now let's add some new values to the set. Here we add `"this value"` and `"new value"`, but `"this value"` has been added twice.

In [8]:
%%bash
redis-cli

sadd set-key "this value"
sadd set-key "new value"
sadd set-key "this value"

1
1
0


It's interesting that outputs looks like it returns `1` if the operation was successful and `0` if not. In our case the last `sadd` tried to add an already existing element.

Now, using `smembers`, we can access the set we have just created.

In [9]:
%%bash 
redis-cli
smembers set-key

this value
new value


Some practice for `sismember` and `srem` in one cell.

In [10]:
%%bash
redis-cli

sismember set-key "new value"
srem set-key "new value"
sismember set-key "new value"

1
1
0


Before removing `sismember` it returns `1`, after removing it returns `0`. The middle `1` is the output of the `srem` command - it signals that everything has been successfully completed.

Don't forget to delete the set you just created, to ensure the same execution for this example and to exclude useless garbage in our reddis instance.

In [11]:
%%bash
redis-cli
del set-key

1
